# Inferencia de las fuerzas moleculares de los elementos del dataset QM9

## Importación de módulos

In [5]:
import torch
import numpy as np
import pandas as pd
import os
from scripts.qm9 import QM9

## Definición de directorios

In [6]:
path_splits_npy = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/splits.npz"

# Ruta al dataset que uso torchmd-net
path_torchmdnet_dataset = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/dataset_qm9.pt"

# Ruta al modelo entrenado
path_trained_model="/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/epoch=669-val_loss=0.0003-test_loss=0.0064.ckpt"

## Cargar modelo

In [7]:
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True)

## Cargar database

In [8]:
# Cargar archivo del dataset que utiliza torchmd-net
torchmdnet_dataset = torch.load(path_torchmdnet_dataset)

## Inferencias individuales

In [9]:
#número de indice de molecula
num_mol=10

In [15]:
molecula=torchmdnet_dataset[num_mol]

z_sample = molecula.z
pos_sample = molecula.pos
n_energy_sample = molecula.y
# Hacer la inferencia
n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
# Transformar la inferencia
#n_energy_inferred = float(n_energy_inferred)
#n_energy_sample = float(n_energy_sample)       

In [16]:
print(n_energy_inferred) 

tensor([[-4184.7759]], grad_fn=<AddBackward0>)


In [20]:
n_forces_inferred=np.array(n_forces_inferred)
print(n_forces_inferred)
print(type(n_forces_inferred))

tensor([[-0.2084, -0.7185,  0.0756],
        [ 0.0928,  1.2006, -0.0136],
        [ 0.3410, -0.0847, -0.1525],
        [-0.0606, -0.1645,  0.0233],
        [ 0.1452, -0.2372,  0.0919],
        [ 0.0259, -0.2323, -0.1782],
        [-0.3359,  0.2367,  0.1535]], grad_fn=<NegBackward0>)
<class 'numpy.ndarray'>


## Creación de un nuevo database con fuerzas inferidas

In [ ]:


# Cargar archivo splits
splits_data = np.load(path_splits_npy)

In [ ]:
# Acceder a la matriz específica dentro del archivo npz
splits = splits_data['idx_test'] 

# Indexar el dataset con los índices de splits
dataset_torchmdnet_test = torchmdnet_dataset[splits]

In [ ]:
if not os.path.exists('data/inferencias_test.dat'):
    with open('data/inferencias_test.dat', 'w') as file:
        for molecula in dataset_torchmdnet_test:
            z_sample = molecula.z
            pos_sample = molecula.pos
            n_energy_sample = molecula.y
            # Hacer la inferencia
            n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
            # Transformar la inferencia
            n_energy_inferred = float(n_energy_inferred)
            n_energy_sample = float(n_energy_sample)        
            # Guardar los datos en el archivo
            file.write(f"{n_energy_sample} {n_energy_inferred}\n")
else:
    print("El archivo 'inferencias_test.dat' ya existe. No se ha realizado ninguna acción.")